# key notes 

## data sources:
1. reddit scraping while possible is probably too noisey to be useful

2. yahoo scraping can be done by scraping the wayback machine recursively use the article tag with role=article (the rss vertion of it dosent have enough saves in waybckmachine)

3. wikipedia scraping is very clean data but unfortunatly there isnt alot of changes so we are again data fin

## train strategy 
###### 1. seperate data into diffrent segments based on how frequantly it gets updated 
2. make a nested training loop that updates based on that frequncy. (this helps preventing needles recalculations)
3. use an lstm on the price data that is prompted with extra information from the other segments

In [24]:
import tensorflow as tf
import numpy as np

"""
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.settings import Settings
from scrapy_wayback_machine import WaybackMachineMiddleware
"""
import concurrent
import datetime

In [25]:
#toy unbatched exmple
tf.config.run_functions_eagerly(False)
N=3
big=tf.data.Dataset.range(N)
r=tf.data.Dataset.random().map(lambda x: 4+x%20).take(N)
small=tf.data.Dataset.zip((big,r))
small=small.map(lambda x,r:tf.range(x,x+r))
small=small.map(lambda x:tf.data.Dataset.from_tensor_slices(x).batch(4)).prefetch(tf.data.AUTOTUNE)
for elem in small:
    #big loop updates HERE
    print(f"len={len(elem)}")
    for x in elem:
        #small loop updates HERE 
        print(x)
tf.config.run_functions_eagerly(True)

len=6
tf.Tensor([0 1 2 3], shape=(4,), dtype=int64)
tf.Tensor([4 5 6 7], shape=(4,), dtype=int64)
tf.Tensor([ 8  9 10 11], shape=(4,), dtype=int64)
tf.Tensor([12 13 14 15], shape=(4,), dtype=int64)
tf.Tensor([16 17 18 19], shape=(4,), dtype=int64)
tf.Tensor([20 21], shape=(2,), dtype=int64)
len=4
tf.Tensor([1 2 3 4], shape=(4,), dtype=int64)
tf.Tensor([5 6 7 8], shape=(4,), dtype=int64)
tf.Tensor([ 9 10 11 12], shape=(4,), dtype=int64)
tf.Tensor([13], shape=(1,), dtype=int64)
len=5
tf.Tensor([2 3 4 5], shape=(4,), dtype=int64)
tf.Tensor([6 7 8 9], shape=(4,), dtype=int64)
tf.Tensor([10 11 12 13], shape=(4,), dtype=int64)
tf.Tensor([14 15 16 17], shape=(4,), dtype=int64)
tf.Tensor([18 19 20], shape=(3,), dtype=int64)


In [26]:
tf.data.Dataset.random().take(1).get_single_element()

<tf.Tensor: shape=(), dtype=int64, numpy=418894321>

In [27]:
import requests as req
import json 
import bs4
import pandas as pd
import os

In [28]:
test_url="https://finance.yahoo.com/quote/VZ/"
cdx_url_template = ('http://web.archive.org/cdx/search/cdx?url={url}'
                    '&output=json')#&fl=timestamp,original,statuscode,digest') 

#test_cdx=cdx_url_template.format(url=test_url)

In [29]:
def get_time_df(url):
    with req.get(cdx_url_template.format(url=url)) as r:
        j=r.content
        j=json.loads(j)
    return pd.DataFrame(j[1:],columns=j[0])#.set_index('timestamp')
time_df=get_time_df(test_url)
#int.from_bytes(bytes(time_df["digest"][0],"ascii") ,'little')
time_df

,urlkey,timestamp,original,mimetype,statuscode,digest,length
0,"com,yahoo,finance)/quote/vz",20160516160413,http://finance.yahoo.com:80/quote/VZ,text/html,301,KAV73IUWIQTFXK7YQBGXBLIOEZ6QGEAN,716
1,"com,yahoo,finance)/quote/vz",20160516160413,http://finance.yahoo.com:80/quote/VZ/,text/html,404,GJ32S3AQF57PJWQTMANB7P5FOJ2VSC4S,996
2,"com,yahoo,finance)/quote/vz",20160517182416,http://finance.yahoo.com:80/quote/VZ,text/html,301,L5ZQ2GB52XBRPQNDA7DXTKV7LBS22CJI,725
3,"com,yahoo,finance)/quote/vz",20160517182416,http://finance.yahoo.com:80/quote/VZ/,text/html,404,GJ32S3AQF57PJWQTMANB7P5FOJ2VSC4S,1001
4,"com,yahoo,finance)/quote/vz",20160517190450,http://finance.yahoo.com:80/quote/vz/,text/html,404,GG7JELIWXCGDX2M7ABDOZX5SKZLEBPCJ,922
...,...,...,...,...,...,...,...
845,"com,yahoo,finance)/quote/vz",20221010165659,https://finance.yahoo.com/quote/VZ/,text/html,200,ZV4FFKK24ZJUYI7B7LURWXQ2A4F3OHAW,200953
846,"com,yahoo,finance)/quote/vz",20221010165701,https://finance.yahoo.com/quote/VZ,text/html,200,MN4GJSQVHVWYNQ5VQ7ZL4YPMJIRMRQVZ,200470
847,"com,yahoo,finance)/quote/vz",20221017211044,https://finance.yahoo.com/quote/VZ,text/html,200,QVX43XJQH2BEGRFVAYXLOPSG2KNH2XU2,202190
848,"com,yahoo,finance)/quote/vz",20221018064802,http://finance.yahoo.com/quote/VZ/,text/html,200,OHES44XCBETVTDK7WEDZD4BVFSOC435P,166431


In [30]:
wayback="https://web.archive.org/web/"
def get_wayback_url(row):
    return wayback+row["timestamp"]+"/"+row["original"]
req.get(get_wayback_url(time_df.iloc[0]))

<Response [404]>

In [31]:
def valid_code(code):
    if(code!="-" and 200<=int(code)<300):
        return True
    return False
valid_time=time_df[time_df["statuscode"].map(valid_code)]
valid_time["statuscode"].value_counts()

200    763
Name: statuscode, dtype: int64

In [32]:
data=valid_time[valid_time["timestamp"].map(lambda x: int(x[0:6])>=201910)]
data["timestamp"].value_counts().value_counts()

1    159
Name: timestamp, dtype: int64

In [33]:
#0
url=get_wayback_url(data.iloc[0])
print(url)
with req.get(url) as r:
    b=bs4.BeautifulSoup(r.content)
    print(r.url==url)

https://web.archive.org/web/20191012054117/https://finance.yahoo.com/quote/VZ
True


In [34]:
#not all of them
c=0
for x in b.find_all("li",class_="js-stream-content"):
    c+=1
    print (f"{c}  {x.text}:\n{x.find(href=True)['href']}+\n")
    assert "href" in str(x)

1  Yahoo Finance Video5G could be 'next great secular growth story'Amanda Agati, chief investment strategist at PNC, gives Yahoo Finance insight on why investors could benefit from avoiding absolute-dividend payers and instead focus on dividend growers, such as tech and health care. Agati explains that specialty real estate investment trusts (REITs) are also worth watching because of their exposure to 5G, which she says is the "next great secular growth story.":
/web/20191012054117/https://finance.yahoo.com/video/5g-could-next-great-secular-143305106.html+

2  Yahoo Finance VideoVerizon Media CEO Guru Gowrappan on the push to 5GTechCrunch’s Disrupt Conference kicked off in San Francisco. Yahoo Finance’s Melody Hahm caught up with Verizon Media CEO Guru Gowrappan to discuss the road ahead for the company and the push to 5G.:
/web/20191012054117/https://finance.yahoo.com/video/verizon-media-ceo-guru-gowrappan-171929745.html+

3  Benzinga5G Will Revolutionize Every Industry ... But Only I

In [36]:
def show_articles(b):
    c=0
    seen=[]
    good=[]
    for x in b.find_all("li",class_="js-stream-content"):
        for x2 in x.find_all(href=True):
            href=x2["href"]
            if(href in seen):
                print("seen") 
            seen.append(href)
            c+=1
            z="*"
            if "https://web.archive.org/" not in href:
                href="https://web.archive.org/"+href
                z=""
            print(f"{c}   {x2.parent.text}:::")
            #print(x2.parent.parent.text)
            if ("finance.yahoo.com/news/" in str(href)):
                good.append(href)
                print("!!!!!!!!!!!!valid!!!!!!!!!!!!!")
                print(f"{z} {href}")
    return good
links=show_articles(b)

1   5G could be 'next great secular growth story':::
2   Verizon Media CEO Guru Gowrappan on the push to 5G:::
3   5G Will Revolutionize Every Industry ... But Only If It Has Blockchain Security:::
!!!!!!!!!!!!valid!!!!!!!!!!!!!
 https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/5g-revolutionize-every-industry-only-182514495.html
4   BUILT BY GIRLS, Speck And Verizon Unveil Winning Designs From Make My Case Competition:::
!!!!!!!!!!!!valid!!!!!!!!!!!!!
 https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/built-girls-speck-verizon-unveil-160000329.html
5   Trader Toolkit: Cash Flow:::
!!!!!!!!!!!!valid!!!!!!!!!!!!!
 https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/trader-toolkit-cash-flow-131805890.html
6   Verizon Is Ready for an Upside Breakout, Add to Longs:::
7   AT&T, Verizon Race to Shore Up Service Threatened by PG&E Outage:::
!!!!!!!!!!!!valid!!!!!!!!!!!!!
 https://web.archive.org//web/20191012054117/https

In [37]:
a="https://web.archive.org/"+"/web/20160727193411/http://finance.yahoo.com/m/9a736299-9075-3caf-b0cb-1466b222214f/ss_t-mobile%E2%80%99s-legere-predicts.html"
print(a)

https://web.archive.org//web/20160727193411/http://finance.yahoo.com/m/9a736299-9075-3caf-b0cb-1466b222214f/ss_t-mobile%E2%80%99s-legere-predicts.html


In [38]:
"rss" in str(b)

False

In [39]:
RP="_________"
def make_addres(url):
    addres=url.replace(":",RP)
    addres=os.path.join("archive",addres)
    return addres

def write_url(url):
    addres=make_addres(url)
    os.makedirs(addres)
    assert RP not in url
    with req.get(url) as r:
        if(url==r.url):
            with open(os.path.join(addres,"content.html"),"wb") as f:
                f.write(r.content)
                return r.content
        with open(os.path.join(addres,"redirect"),"w") as f:
                addres=make_addres(r.url)
                f.write(addres)
        
        os.makedirs(addres)
        with open(os.path.join(addres,"content.html"),"wb") as f:
                f.write(r.content)
                return r.content
           

#write_url("https://web.archive.org/web/20191012054117/https://finance.yahoo.com/quote/VZ")

In [40]:
def archive_get(url):
    addres=make_addres(url)
    try:
        files=next(os.walk(addres))[-1]
    except StopIteration:
        return write_url(url)
    if files[0]=="redirect":
        with open(os.path.join(addres,"redirect")) as f:
            addres=f.read()
    with open(os.path.join(addres,"content.html"),"rb") as f:
            return f.read()
#archive_get("https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/built-girls-speck-verizon-unveil-160000329.html")

In [44]:
def get_articles(b):
    for x in b.find_all("li",class_="js-stream-content"):
        for x2 in x.find_all(href=True):
            href=x2["href"]
            if "https://web.archive.org/" not in href:
                href="https://web.archive.org/"+href
            yield (x2.text,x2.parent.parent.text,href)
            
def handle_link(url):
    if "finance.yahoo.com/news/" not in url:
        return None
    try:
        with req.get(url) as r:
            return (bs4.BeautifulSoup(r.content).find("article").text,r.url)
    except Exception:
        print(Exception.__class__.__name__)
        return None
for a in get_articles(b):
    print(a[-1])

https://web.archive.org//web/20191012054117/https://finance.yahoo.com/video/5g-could-next-great-secular-143305106.html
https://web.archive.org//web/20191012054117/https://finance.yahoo.com/video/verizon-media-ceo-guru-gowrappan-171929745.html
https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/5g-revolutionize-every-industry-only-182514495.html
https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/built-girls-speck-verizon-unveil-160000329.html
https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/trader-toolkit-cash-flow-131805890.html
https://web.archive.org//web/20191012054117/https://finance.yahoo.com/m/400a414c-f65c-34c1-802d-2853984724d6/verizon-is-ready-for-an.html
https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/t-verizon-race-shore-threatened-164706367.html
https://web.archive.org//web/20191012054117/https://finance.yahoo.com/news/verizon-ceo-5g-will-transform-the-world-154708054.html
ht

In [42]:
def thread_distribute(l,f,n=1):
    #print(type(l))
    if(n==0):
        return f(l)
    
    with concurrent.futures.ThreadPoolExecutor() as executor: 
        l=[executor.submit(lambda x: thread_distribute(x,f,n-1),x) for x in l]
        l= [x.result() for x in l]
    return l

thread_distribute([range(5) for i in range(2)],lambda x: range(x),2)

[[range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)],
 [range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)]]

In [ ]:
data.iloc[::-1]

In [ ]:
for a in get_articles(b):
    print(f"{a[0]} \n\n {a[1]} \n*********************\n")

In [13]:
file_names=["headline","summary","link"]
def save_page(row,save_adress=os.path.join("archive","VZ")):
    url=get_wayback_url(row)
    with req.get(url) as r:
        ans=get_articles(bs4.BeautifulSoup(r.content))
    
    for n,article in enumerate(ans):
        folder=os.path.join(save_adress,row["timestamp"],str(n))
        os.makedirs(folder)
        for i in range(3):
            with open(os.path.join(folder,file_names[i]),mode="+w") as f:
                f.write(article[i])

#save_page(data.iloc[-1])

In [ ]:
thread_distribute(data.iloc[::-1].iloc,save_page)

In [ ]:
a=next(os.walk(os.path.join("archive","VZ")))
len(a[1]),len(data)

In [16]:
def colect_stock(ticker):
    base_url=f"https://finance.yahoo.com/quote/{ticker}/"
    
    data=get_time_df(base_url)
    data=time_df[data["statuscode"].map(valid_code)]
    data=data[data["timestamp"].map(lambda x: int(x[0:6])>=201910)]
    
    thread_distribute(data.iloc[::-1].iloc,lambda x: save_page(x,os.path.join("archive",ticker)))
    
colect_stock("VZ")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'archive\\VZ\\20221018064915\\0'

In [ ]:
for stock in ("T","TMUS"):
    colect_stock(stock)

NameError: name 'data' is not defined